In [1]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
from utils import (
    create_client,
    create_schema,
    create_collection,
    load_images,
    create_index,
    search_by_image,
    search_by_text,
    draw_images,
)
import time
import matplotlib.pyplot as plt
from pymilvus import MilvusClient
from clip_embeddings_generator import ClipEmbeddingsGenerator

Índices que vamos a testear:

### 1. IVF_FLAT (Inverted File System with Flat Search)
Es una variante del índice de cuantización vectorial que primero divide el espacio del vector en regiones utilizando un algoritmo de clustering como K-means. Cada vector se asigna al clúster más cercano y se almacena su representación exacta. Durante la búsqueda, sólo se explora el clúster (o los clústeres) más cercano al vector de consulta, reduciendo significativamente el número de comparaciones necesarias.

**Parámetros**:

- `nlist`: Especifica el número de clústeres a utilizar. Un valor más alto puede mejorar la precisión de la búsqueda a costa de utilizar más memoria y aumentar el tiempo de indexación. Un valor de `32` está equilibrado para un conjunto de datos de tamaño medio como el de 229 imágenes.

### 2. HNSW (Hierarchical Navigable Small World)

Construye un grafo de múltiples capas donde cada capa es un grafo "de mundo pequeño". La capa superior tiene menos nodos y sirve como entrada rápida a las capas inferiores que contienen más nodos. Durante la inserción, cada nuevo vector se conecta a los nodos existentes de manera que se mantenga la propiedad del mundo pequeño, asegurando que cualquier nodo pueda ser alcanzado desde cualquier otro en un pequeño número de saltos. Para la búsqueda, se navega desde la capa superior hacia la inferior, aprovechando estos atajos para encontrar rápidamente los vecinos más cercanos del vector de consulta.

**Parámetros**:

- `M`: Controla el número de conexiones de borde que cada nodo tiene en el grafo. Un valor más alto puede mejorar la precisión pero aumenta el uso de memoria.
- `efConstruction`: Controla el tamaño de la cola dinámica usada durante la construcción del índice, afectando la calidad del grafo. Un valor más alto puede mejorar la precisión pero aumenta el tiempo de construcción del índice.

### 3. Métricas de Distancia
- `L2` (Distancia euclídea): Calcula la distancia geométrica tradicional entre dos puntos. Es efectiva cuando la magnitud de los vectores es importante.
- `IP` (Inner Product): Calcula la similitud como el producto punto de dos vectores, lo cual es útil para medir similitud en términos de ángulo, independientemente de la magnitud.
- `COSINE` (


In [ ]:
# Configuraciones de índices y métricas para probar
index_metrics = [
    {"index_type": "FLAT", "metric_type": "COSINE", "params": {}},
    {"index_type": "FLAT", "metric_type": "L2", "params": {}},
    {"index_type": "FLAT", "metric_type": "IP", "params": {}},
    {"index_type": "IVF_FLAT", "metric_type": "COSINE", "params": {"nlist": 4}},
    {"index_type": "IVF_FLAT", "metric_type": "COSINE", "params": {"nlist": 8}},
    {"index_type": "IVF_FLAT", "metric_type": "COSINE", "params": {"nlist": 16}},
    {
        "index_type": "HNSW",
        "metric_type": "COSINE",
        "params": {"M": 2, "efConstruction": 500},
    },
    {
        "index_type": "HNSW",
        "metric_type": "IP",
        "params": {"M": 4, "efConstruction": 500},
    },
    {
        "index_type": "HNSW",
        "metric_type": "IP",
        "params": {"M": 8, "efConstruction": 500},
    },
]

# Crear cliente y esquema
client: MilvusClient = create_client()
schema = create_schema()

# Inicializar el generador de embeddings
embedder = ClipEmbeddingsGenerator()

Failed to create new connection using: 97e1fbd9096044fe9bd2e9f421e9d945


In [ ]:
image_dir = "images/"
image_paths = [
    os.path.join(image_dir, img)
    for img in os.listdir(image_dir)
    if img.endswith(".png")
]

In [ ]:
# Crea y carga imágenes en diferentes colecciones para cada configuración de índice
for config in index_metrics:
    collection_name = f"avatar_{config['index_type']}_{config['metric_type']}"
    create_collection(client, schema, collection_name)
    load_images(embedder, client, collection_name)
    start_time = time.perf_counter()
    create_index(client, collection_name, config=config)
    client.load_collection(collection_name=collection_name)
    elapsed_time_load = time.perf_counter() - start_time
    print(f"Collection: '{collection_name}', config = {config['params']}")
    print(f"Time to load: {elapsed_time_load}")
    
    # Ejemplo de búsqueda y medición de tiempo
    text_query = "An old cat"
    start_time = time.perf_counter()
    for _ in range(10):
        search_results = search_by_text(client, collection_name, embedder, text_query)
    elapsed_time = time.perf_counter() - start_time
    
    
    print(f"Time to search {elapsed_time}")
    result_ids = [result["id"] for result in search_results[0]]
    image_paths_result = [image_paths[result_id] for result_id in result_ids]
    result_titles = [f"{result['id']} - dist: {result['distance']:.2f}" for result in search_results[0]]
    
    draw_images(image_paths_result, result_titles)
    plt.show()
    
    # Liberar la colección después de la prueba
    client.release_collection(collection_name)